## таблица выход из створа

критерии выхода:  
- 100м = 0,1м
- 200м = 0,2м
- 300м = 0,3м

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from shapely.geometry import Point, MultiPoint, Polygon, LineString
from math import sqrt, acos, cos, sin, degrees, radians

In [69]:
# теперь загрузим таблицу и поработаем с ней
workdir = Path(r'D:\work\2022_m\220_man-van\test')
strain_tab = pd.read_excel(workdir / 'strain.xlsx')

In [70]:
strain_tab

,id,x,y,z,l,type
0,ПС_Мангазея,557349.94,7388274.42,19.52,40.24,strain
1,1,557342.11,7388234.95,17.89,106.61,strain
2,2,557409.02,7388151.95,16.39,187.55,susp
3,3,557526.72,7388005.93,15.58,135.84,strain
4,4,557662.46,7388000.73,18.41,147.95,strain
...,...,...,...,...,...,...
604,604,604092.15,7513441.75,33.82,121.73,strain
605,605,604017.81,7513538.14,32.59,56.54,strain
606,606,603964.49,7513556.95,33.74,66.35,strain
607,607,603920.39,7513507.38,34.42,29.73,strain


In [71]:
strain_tab.assign(dist=np.nan, norma=np.nan, ot=np.nan)   # создаем колонки 

,id,x,y,z,l,type,dist,norma,ot
0,ПС_Мангазея,557349.94,7388274.42,19.52,40.24,strain,NaN,NaN,NaN
1,1,557342.11,7388234.95,17.89,106.61,strain,NaN,NaN,NaN
2,2,557409.02,7388151.95,16.39,187.55,susp,NaN,NaN,NaN
3,3,557526.72,7388005.93,15.58,135.84,strain,NaN,NaN,NaN
4,4,557662.46,7388000.73,18.41,147.95,strain,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
604,604,604092.15,7513441.75,33.82,121.73,strain,NaN,NaN,NaN
605,605,604017.81,7513538.14,32.59,56.54,strain,NaN,NaN,NaN
606,606,603964.49,7513556.95,33.74,66.35,strain,NaN,NaN,NaN
607,607,603920.39,7513507.38,34.42,29.73,strain,NaN,NaN,NaN


In [72]:
for i in range(len(strain_tab)):
    #print(strain_tab.loc[i, 'id'])

    # check if not Strain
    if strain_tab.loc[i, 'type'] == 'susp':
        stvor = LineString([(strain_tab.loc[i-1, 'x'], strain_tab.loc[i-1, 'y']), (strain_tab.loc[i+1, 'x'], strain_tab.loc[i+1, 'y'])])
        strain_tab.loc[i, 'dist'] = round(Point((strain_tab.loc[i, 'x'], strain_tab.loc[i, 'y'])).distance(stvor), 2)
        
        norma = 0
        if strain_tab.loc[i, 'l'] < 200: norma = 0.1
        elif 200 < strain_tab.loc[i, 'l'] < 300: norma = 0.2
        else: norma = 0.3
        strain_tab.loc[i, 'norma'] = norma
        
        if strain_tab.loc[i, 'dist'] > norma:
            strain_tab.loc[i, 'ot'] = strain_tab.loc[i, 'dist'] - norma


In [73]:
strain_tab[:-10]

,id,x,y,z,l,type,dist,norma,ot
0,ПС_Мангазея,557349.94,7388274.42,19.52,40.24,strain,NaN,NaN,NaN
1,1,557342.11,7388234.95,17.89,106.61,strain,NaN,NaN,NaN
2,2,557409.02,7388151.95,16.39,187.55,susp,0.00,0.1,NaN
3,3,557526.72,7388005.93,15.58,135.84,strain,NaN,NaN,NaN
4,4,557662.46,7388000.73,18.41,147.95,strain,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
594,594,603535.14,7510867.27,35.73,309.94,susp,0.05,0.3,NaN
595,595,603563.51,7511175.91,34.59,295,susp,0.07,0.2,NaN
596,596,603590.38,7511469.68,33.91,334.78,susp,0.10,0.3,NaN
597,597,603621.08,7511803.05,27.93,177.8,susp,0.06,0.1,NaN


In [74]:
# save tab
strain_tab.to_csv(workdir / '220_stvor.txt', sep='\t', encoding = 'cp1251')